In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as linalg
import scipy.stats as stats
import torch
from statsmodels.api import OLS
import matplotlib.pyplot as plt

d = 3

gscale = 5.8454   ## from vegan

## RDA

![](RDA.png)

### Simple RDA
$Y$在$X$上进行多元回归($y_{ii}=\beta_1x_{i1}+\beta_2x_{i2}+...$)，得到：拟合值矩阵 $\hat{Y}^{n \times p}=X^{n \times m}B^{m \times p}=X(X'X)^{-1}X'Y$、残差矩阵$Y_{res}=Y-\hat{Y}$

* 对$\hat{Y}$进行PCA分析，得到约束轴(constrained/Canonical axes)$RDA_i$上展示的信息 ($Y$ 中被 $X$ 解释的部分)
* 对$Y_{res}=Y-\hat{Y}$ 进行PCA，得到非约束轴(unconstrained/non-Canonical axes)$PC_i$上展示的信息 ($Y$ 中被 residuals 解释的部分)

----------------------

1. $U$ 中每一列为 $\hat{Y}$ 的特征向量
2. $F=YU$ 原样本在轴上的坐标 = ordination in the space of variables Y
3. $Z=\hat{Y}U=XBU$ Fitted样本在轴上的坐标 = ordination in the space of variables X

----------------------
**scaling 1**: eigenvectors normalized to length 1 ; **Biplot scores** $BS_1=Var(Y)^{-1/2}cor(X,Z)\Lambda^{1/2}$ 即: $cor(x,Z)\sqrt{\lambda_k/Var(Y)}$

**scaling 2**: eigenvectors normalized to length $\sqrt{\lambda_k}$ ; **Biplot scores** $BS_2=R_{XZ}=cor(X,\hat{Y}U)$

注：**scaling type 1**的结果乘上 $\Lambda^{-1/2}$ 就是 **scaling type 2**的结果，例如$U\Lambda^{-1/2}$、$Z\Lambda^{-1/2}$、$F\Lambda^{-1/2}$，$Z\Lambda^{-1/2}\Lambda^{1/2}U'=\hat{Y}$

----------------------

### Partial RDA
回归时加上 Conditioning variables W 矩阵

### db-RDA
原始数据进行PCoA，将PCoA排序轴上的 Site scores 作为 Response Matrix $Y$


## RDA轴的数量


![](RDA_axis.png)

轴的总数量为(n_sample-1)，其中约束轴数目为(explain_x_level)，余下为非约束轴；其中 explain_x_level = quantitative_x数目 + (categorical_x中类别数-1)


In [2]:
Y = np.matrix([                           ##  10 Sample * 9 var
    [1,0,0,0,0,0,2,4,4],
    [0,0,0,0,0,0,5,6,1],
    [0,1,0,0,0,0,0,2,3],
    [11,4,0,0,8,1,6,2,0],
    [11,5,17,7,0,0,6,6,2],
    [9,6,0,0,6,2,10,1,4],
    [9,7,13,10,0,0,4,5,4],
    [7,8,0,0,4,3,6,6,4],
    [7,9,10,13,0,0,6,2,0],
    [5,10,0,0,2,4,0,1,3]
])



X = np.matrix([                           ##  10 Sample * 4 var
    [1,0,1,0],
    [2,0,1,0],
    [3,0,1,0],
    [4,0,0,1],
    [5,1,0,0],
    [6,0,0,1],
    [7,1,0,0],
    [8,0,0,1],
    [9,1,0,0],
    [10,0,0,1]
])


Y.shape,X.shape

((10, 9), (10, 4))

In [3]:
Yc = Y - Y.mean(0)
Xc = X - X.mean(0)

In [4]:
## https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html
Y_hat = pd.DataFrame(Yc).apply(lambda y: OLS(y, X).fit().fittedvalues)
Y_hat = np.matrix(Y_hat)
Y_hat.shape

(10, 9)

In [5]:
S_hat = Y_hat.T @ Y_hat / (10-1)
L, U = linalg.eigh(S_hat)
L = L[::-1]                
U = U[:,::-1]

U = U[:,:d]
L = L[:d]

U[:,0] = -U[:,0]

L,U      ## eigenValues  and  Species scores

(array([75.6049424 , 27.68145497, 10.60434337]),
 array([[ 0.30903799,  0.61947722,  0.26645171],
        [ 0.19857338,  0.40557273, -0.74910251],
        [ 0.73539401, -0.1724343 ,  0.21645813],
        [ 0.54217697, -0.19025599, -0.24569748],
        [-0.10727289,  0.49611898,  0.20935378],
        [-0.06196403,  0.19389925, -0.25802629],
        [ 0.10093152,  0.29757154,  0.32969986],
        [ 0.05625029, -0.11618219,  0.19015024],
        [-0.04007043,  0.00705125, -0.07322475]]))

In [6]:
U @ np.diag(np.sqrt(L))    

array([[ 2.68711938,  3.25926592,  0.86768161],
       [ 1.72661745,  2.13384666, -2.4394007 ],
       [ 6.39433197, -0.90723146,  0.70488097],
       [ 4.71428854, -1.00099707, -0.80009691],
       [-0.93274961,  2.61023916,  0.68174616],
       [-0.53878408,  1.02016538, -0.84024481],
       [ 0.87761073,  1.56561815,  1.07364486],
       [ 0.48910251, -0.61127129,  0.61921114],
       [-0.34841678,  0.03709888, -0.23845136]])

In [7]:
F = Yc @ U                    ## Site scores  (unscaled)   ---- Question: Vegan 到底scale了啥。。
F

matrix([[-7.11396328, -5.84232486,  1.28482103],
        [-6.88749483, -5.8226056 ,  2.60744363],
        [-7.4987211 , -6.42605931, -0.69720862],
        [-3.69792981,  7.53203487,  3.60112998],
        [13.86258795, -0.93904473,  4.00928073],
        [-3.57908317,  7.64056054,  1.72903787],
        [11.98835943, -1.71274288, -0.62075211],
        [-3.76990526,  4.64321577, -1.34685266],
        [11.38117299, -1.27853233, -3.65647934],
        [-4.68502292,  2.20549854, -6.91042052]])

In [8]:
Z = Y_hat @ U                  ## Site constraints  (unscaled)  ,   F/Z同Vegan    ---- Question: Vegan 到底scale了啥。。
Z

matrix([[-6.88340343, -5.35567448,  2.79582174],
        [-7.1667264 , -6.03032992,  1.06501868],
        [-7.45004937, -6.70498537, -0.66578438],
        [-3.08301638,  7.52929377,  4.46063285],
        [12.97735274,  0.03920424,  3.37228922],
        [-3.64966232,  6.17998287,  0.99902673],
        [12.41070679, -1.31010665, -0.0893169 ],
        [-4.21630826,  4.83067198, -2.46257939],
        [11.84406085, -2.65941754, -3.55092302],
        [-4.78295421,  3.48136109, -5.92418551]])

In [9]:
## cor(X_col,Z_col)      == BS2 --- 似乎下文Vegan操作后得到的是BS2， 不过为何Vegan结果只有 X1-3？？
R_XZ = [[ stats.pearsonr(np.array(X[:,i]).squeeze(),np.array(Z[:,j]).squeeze())[0] for j in range(Z.shape[1])]for i in range(X.shape[1])]
R_XZ

[[0.4178468826945435, 0.48976639185235893, -0.7652011918668016],
 [0.9849445343085712, -0.1718314070375067, -0.01892701485133498],
 [-0.5687692182264668, -0.7909280342008871, 0.22568655501197524],
 [-0.3892963614028966, 0.9005789937489972, -0.1934058401820108]]

In [10]:
BS = R_XZ @ np.diag(np.sqrt(L)) *(Y.var() ** (-0.5) )     #### BS1 
BS

array([[ 0.93378149,  0.66227206, -0.64042832],
       [ 2.20110047, -0.23235392, -0.0158408 ],
       [-1.27105451, -1.06950895,  0.18888635],
       [-0.86997833,  1.21778121, -0.16186929]])

作图时候，一般用箭头画BS（对应X_vars），用点画F（Y_vars 的 Site scores）

## 关于Vegan

```R
library(vegan)
Y = matrix( c(1, 0, 0, 0, 0, 0, 2, 4, 4, 0, 0, 0, 0, 0, 0, 5, 6, 1, 0, 1, 0, 0, 0, 0, 0, 2, 3, 11, 4, 0, 0, 8, 1, 6, 2, 0, 11, 5, 17, 7, 0, 0, 6, 6, 2, 9, 6, 0, 0, 6, 2, 10, 1, 4, 9, 7, 13, 10, 0, 0, 4, 5, 4, 7, 8, 0, 0, 4, 3, 6, 6, 4, 7, 9, 10, 13, 0, 0, 6, 2, 0, 5, 10, 0, 0, 2, 4, 0, 1, 3), ncol = 9,byrow = TRUE)
colnames(Y) = c('V1','V2','V3','V4','V5','V6','V7','V8','V9')

X = matrix( c(1, 0, 1, 0, 2, 0, 1, 0, 3, 0, 1, 0, 4, 0, 0, 1, 5, 1, 0, 0, 6, 0, 0, 1, 7, 1, 0, 0, 8, 0, 0, 1, 9, 1, 0, 0, 10, 0, 0, 1), ncol = 4,byrow = TRUE)
colnames(X) = c('E1','E2','E3','E4')

rda <- rda(Y ~ X, center = TRUE, scale= FALSE)
summary(rda, axes = 2,scaling=0)
ordiplot(rda)
```

RDA对象中数据默认不Scale，但是在Summary或Plot时又会默认进行scaling="species"(i.e.spe*scale，其它乘 General scaling constant)

![](vegan_summary_scale.png)


```
....
Scaling 0 for species and site scores
* Both are 'unscaled' or as they are in the result
* General scaling constant of scores:  5.8454
....
Scaling 1 for species and site scores
* Sites are scaled proportional to eigenvalues
* Species are unscaled: weighted dispersion equal on all dimensions
* General scaling constant of scores:  5.8454
....
Scaling 2 for species and site scores
* Species are scaled proportional to eigenvalues
* Sites are unscaled: weighted dispersion equal on all dimensions
* General scaling constant of scores:  5.8454
```

In [11]:
# str = 't(matrix( c('
# for i in range(X.shape[0]):
#     for j in range(X.shape[1]):
#         str = str + '{}, '.format(X[i,j])
# str = str + '), ncol = {},byrow = TRUE))'.format(X.shape[1])
# str

## 旧Vegan例子

```
## 欧氏距离下的db-RDA capscale() 等效于 rda()
## rda(Y ~ X + Condition(W)) 等于 rda(Y, X, W); X, W can be missing
## DataMatrix ~ ConstrainVar1 + Condition(Var)

data(dune)     ## decostand(dune, method = "hellinger")
data(dune.env)
####################################### Only Data Y = Only PCA
xrda <- rda(dune, center = FALSE, scale= FALSE)
biplot(xrda,type = c("text","points"))  


####################################### With constrains X
crda <- rda(dune ~ ., dune.env, center = FALSE, scale= FALSE) 
ordiplot(crda) 


####################################### With constrains X & condition W
zrda <- rda(dune ~ A1 + Condition(Manure), dune.env, center = FALSE, scale= FALSE) 
ordiplot(zrda) 


#########################  结果说明  #############################

## eig占总体eig的比例
RDA_eig_prop = crda$CCA$eig / crda$tot.chi
PC_eig_prop = crda$CA$eig / crda$tot.chi


## scaled pos
### 默认scaling="species", 即 species scaled by eigenvalues
summary(crda, axes = 2) 
ordiplot(crda, type="n") |>
  points("sites", pch=16, col="grey") |>  
  text("species", pch=10, col="red") |> 
  text("biplot", arrows = TRUE, length=0.05, col="blue") 


## unscaled pos,scaling=0 改为 scaling=2 就如默认 scaled pos 一般
## 尝试但对不上！！ scale(crda$CCA$wa, scale = RDA_eig_prop,center=F)
summary(crda,scaling=0,axes=2)$sites  ## Site scores: 样本点(dune行名)在各轴上的坐标，crda$CCA$wa  ??看Doc crda$CCA$u 才是site坐标
summary(crda,scaling=0,axes=2)$species  ## Species scores: spe(dune列名)在各轴上的坐标，crda$CCA$v 
summary(crda,axes=2)$biplot ## ENV 箭头坐标 = crda$CCA$biplot 

summary(crda,scaling=0,axes=2)$constraints ## Site constraints: 样本点的fitted Site scores，crda$CCA$u
```

